### Sources
- https://shap.readthedocs.io/en/latest/example_notebooks/overviews/An%20introduction%20to%20explainable%20AI%20with%20Shapley%20values.html
- https://towardsdatascience.com/interpretable-machine-learning-models-aef0c7be3fd9
- https://github.com/henniedeharder/interpretability-heart/blob/master/Demo_Forest_SHAP_Heart.ipynb

In [2]:
import numpy as np
import shap
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [4]:
"""
Dataset
"""
data = pd.read_csv('csgo_round_snapshots.csv')
print(data.head(10))
print(data.describe())
print(data.shape)

   time_left  ct_score  t_score       map  bomb_planted  ct_health  t_health  \
0     175.00       0.0      0.0  de_dust2         False      500.0     500.0   
1     156.03       0.0      0.0  de_dust2         False      500.0     500.0   
2      96.03       0.0      0.0  de_dust2         False      391.0     400.0   
3      76.03       0.0      0.0  de_dust2         False      391.0     400.0   
4     174.97       1.0      0.0  de_dust2         False      500.0     500.0   
5     114.97       1.0      0.0  de_dust2         False      500.0     500.0   
6      94.97       1.0      0.0  de_dust2         False      295.0     111.0   
7      74.97       1.0      0.0  de_dust2         False      295.0     111.0   
8      54.97       1.0      0.0  de_dust2         False      249.0      89.0   
9     174.97       2.0      0.0  de_dust2         False      500.0     500.0   

   ct_armor  t_armor  ct_money  ...  t_grenade_flashbang  \
0       0.0      0.0    4000.0  ...                  0.0   

In [5]:
"""
Exploratory Data Analysis (EDA) 
"""
cols_meaning = ['round_winner', 'bomb_planted', 'bomb_defused', 'bomb_exploded', 'bomb_planted_in_round', 'bomb_defused_in_round', 'bomb_exploded_in_round', 'bomb_planted_in_match', 'bomb_defused_in_match', 'bomb_exploded_in_match', 'bomb_planted_in_half', 'bomb_defused_in_half', 'bomb_exploded_in_half', 'bomb_planted_in_game', 'bomb_defused_in_game', 'bomb_exploded_in_game', 'bomb_planted_in_map', 'bomb_defused_in_map', 'bomb_exploded_in_map', 'bomb_planted_in_series', 'bomb_defused_in_series', 'bomb_exploded_in_series', 'bomb_planted_in_tournament', 'bomb_defused_in_tournament', 'bomb_exploded_in_tournament', 'bomb_planted_in_season', 'bomb_defused_in_season', 'bomb_exploded_in_season', 'bomb_planted_in_year', 'bomb_defused_in_year', 'bomb_exploded_in_year', 'bomb_planted_in_month', 'bomb_defused_in_month', 'bomb_exploded_in_month', 'bomb_planted_in_day', 'bomb_defused_in_day', 'bomb_exploded_in_day', 'bomb_planted_in_hour', 'bomb_defused_in_hour', 'bomb_exploded_in_hour', 'bomb_planted_in_minute', 'bomb_defused_in_minute', 'bomb_exploded_in_minute', 'bomb_planted_in_second', 'bomb_defused_in_second', 'bomb_exploded_in_second', 'bomb_planted_in_weekday', 'bomb_defused_in_weekday', 'bomb_exploded_in_weekday', 'bomb_planted_in_week', 'bomb_defused_in_week', 'bomb_exploded_in_week', 'bomb_planted_in_quarter', 'bomb_defused_in_quarter', 'bomb_exploded_in_quarter', 'bomb_planted_in_half', 'bomb_defused_in_half', 'bomb_exploded_in_half', 'bomb_planted_in_game', 'bomb_defused_in_game', 'bomb_exploded_in_game', 'bomb_planted_in_map', 'bomb_defused_in_map', 'bomb_exploded_in_map', 'bomb_planted_in_series', 'bomb_defused_in_series', 'bomb_exploded_in_series', 'bomb_planted_in_tournament']


cols_index = [i for i in range(0, len(cols_meaning))]
cols_dict = dict(zip(cols_index, cols_meaning))


In [6]:
"""
Data Preprocessing
"""
y = data['round_winner']
x = data.drop(['round_winner'], axis=1)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [ ]:
"""
Interpretation models
"""

logistic_regression_model = LogisticRegression(random_state=10)



